# Replication Code

John A. Graves

Notes:

-   The original paper uses pairwise conversion formulas. We have verified that we can replicate their results if we do this. However, we use the matrix exponentiation method here.

-   For QALY-like DALYs, the levels are very different (and are a function of number of cycles), but the incremental results are similar and actually yield the same ICER for strategy AB relative to B. Why ? the problem with QALY-like DALYs comes through the deaths. Any strategy that affects death will yield different results relative to our approach. Strategy AB, relative to strategy B, however, just affects the disability weight. So the incremental effect of adding A to B yields the same answer as under our DALY approach—but this will in general not hold if deaths are affected.

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select

Loading required package: Matrix

Attaching package: 'Matrix'

The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Attaching package: 'expm'

The following object is masked from 'package:Matrix':

    expm


Attaching package: 'kableExtra'

The following object is masked from 'package:dplyr':

    group_rows

In [ ]:
fn_r_HD <- function(age) {
  # Access r_HD from the parent frame where this function is called
  r_HD <- get("r_HD", envir = parent.frame())
  r_HD
}

fn_r_HS1 <- function(age) {
  # Access r_HD from the parent frame where this function is called
  r_HS1 <- get("r_HS1", envir = parent.frame())
  r_HS1
}

fn_r_S1H <- function(age) {
  # Access from the parent frame where this function is called
  r_S1H <- get("r_S1H", envir = parent.frame())
  r_S1H
}

fn_r_S1S2 <- function(age) {
  # Access  from the parent frame where this function is called
  r_S1S2 <- get("r_S1S2", envir = parent.frame())
  r_S1S2
}

params1 <- with(params,modifyList(params,list(
    # Natural History Transition Rate Matrix
    m_R = 
      ages %>% map(~({
        mR_SoC = 
          matrix(c(
          -(fn_r_HD(.x)+fn_r_HS1(.x)), fn_r_HS1(.x), 0, fn_r_HD(.x), 0,
          fn_r_S1H(.x),-(fn_r_S1H(.x) + fn_r_S1S2(.x) + fn_r_HD(.x) + hr_S1 * fn_r_HD(.x) - fn_r_HD(.x)),fn_r_S1S2(.x),fn_r_HD(.x),hr_S1 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,-(fn_r_HD(.x) + hr_S2 * fn_r_HD(.x) - fn_r_HD(.x)),fn_r_HD(.x),hr_S2 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,0,0,0,          
          0,0,0,0,0),
          nrow = n_states, 
          ncol = n_states,
          byrow=TRUE, 
          dimnames = list(c(tr_names,ab_names),
                          c(tr_names,ab_names)
          ))
        
        mR_A = 
          matrix(c(
          -(fn_r_HD(.x)+fn_r_HS1(.x)), fn_r_HS1(.x), 0, fn_r_HD(.x), 0,
          fn_r_S1H(.x),-(fn_r_S1H(.x) + fn_r_S1S2(.x) + fn_r_HD(.x) + hr_S1 * fn_r_HD(.x) - fn_r_HD(.x)),fn_r_S1S2(.x),fn_r_HD(.x),hr_S1 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,-(fn_r_HD(.x) + hr_S2 * fn_r_HD(.x) - fn_r_HD(.x)),fn_r_HD(.x),hr_S2 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,0,0,0,          
          0,0,0,0,0),
          nrow = n_states, 
          ncol = n_states,
          byrow=TRUE, 
          dimnames = list(c(tr_names,ab_names),
                          c(tr_names,ab_names)
          ))
        
        mR_B = 
          matrix(c(
          -(fn_r_HD(.x)+fn_r_HS1(.x)), fn_r_HS1(.x), 0, fn_r_HD(.x), 0,
          fn_r_S1H(.x),-(fn_r_S1H(.x) + hr_S1S2_trtB * fn_r_S1S2(.x) + fn_r_HD(.x) + hr_S1 * fn_r_HD(.x) - fn_r_HD(.x)),hr_S1S2_trtB *  fn_r_S1S2(.x),fn_r_HD(.x),hr_S1 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,-(fn_r_HD(.x) + hr_S2 * fn_r_HD(.x) - fn_r_HD(.x)),fn_r_HD(.x),hr_S2 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,0,0,0,          
          0,0,0,0,0),
          nrow = n_states, 
          ncol = n_states,
          byrow=TRUE, 
          dimnames = list(c(tr_names,ab_names),
                          c(tr_names,ab_names)
          ))
        
        mR_AB = 
          matrix(c(
          -(fn_r_HD(.x)+fn_r_HS1(.x)), fn_r_HS1(.x), 0, fn_r_HD(.x), 0,
          fn_r_S1H(.x),-(fn_r_S1H(.x) + hr_S1S2_trtB *  fn_r_S1S2(.x) + fn_r_HD(.x) + hr_S1 * fn_r_HD(.x) - fn_r_HD(.x)),hr_S1S2_trtB * fn_r_S1S2(.x),fn_r_HD(.x),hr_S1 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,-(fn_r_HD(.x) + hr_S2 * fn_r_HD(.x) - fn_r_HD(.x)),fn_r_HD(.x),hr_S2 * fn_r_HD(.x) - fn_r_HD(.x),
          0,0,0,0,0,          
          0,0,0,0,0),
          nrow = n_states, 
          ncol = n_states,
          byrow=TRUE, 
          dimnames = list(c(tr_names,ab_names),
                          c(tr_names,ab_names)
          ))
        
        array(c(as.vector(mR_SoC),
                as.vector(mR_A), 
                as.vector(mR_B),
                as.vector(mR_AB)), 
              dim = c(length(tr_names)+ length(ab_names),length(tr_names)+ length(ab_names),length(tx_names)),
          dimnames = list(c(tr_names,ab_names),c(tr_names,ab_names),tx_names)) %>% 
            apply(.,3,function(x) x, simplify=FALSE) 
        
      }))
    )))

params1 <- with(params1,modifyList(params1,list(
    m_P = m_R %>% transpose() %>% map(~({
      mR_ = .x
      mR_ %>% map(~({
              expm(.x * Delta_t)
         }))
      }))
)))

In [ ]:
trace1 <- 
    with(params1, {
        m_P %>% map( ~ ({
            P = .x
            occ <- s0T
            P %>% map(~({
              occ <<- occ %*% .x
            })) %>% 
            map(~(data.frame(.x))) %>% 
            bind_rows()
        }))
    })  %>% 
    map(~({
        tmp = .x[1,]
        tmp[1,] = params$s0T
        tmp = rbind(tmp,.x)
    }))
tr <- trace1$SoC

In [ ]:
# Life Expectancy (Non-Discounted)
le_ = with(params1,(matrix(c(1,
              1 ,
              1,
              0,
              0)*Delta_t,
            dimnames = list(c(
                c(tr_names,ab_names)
            ), c("DW")))
))

LEt <- trace1 %>% map( ~ ({
    tmp = as.matrix(.x) %*% le_
    tmp 
}))

LE = LEt %>% map(~sum(.x * gen_wcc(params1$omega, method = params1$cycle_correction))) 

# QALYs
qaly_ = with(params1,(matrix(c(u_H,
              u_S1 ,
              u_S2,
              u_D,
              u_D) * Delta_t,
            dimnames = list(c(
                c(tr_names,ab_names)
            ), c("UW")))
))
qaly_ <- 
  with(params1,{
    tx_names %>% map(~({
        if (.x=="A" | .x=="AB") {
          tmp_ <- qaly_
          tmp_[2,1] = u_trtA
          tmp_
      } else qaly_
    }))
  })

QALYt <- trace1 %>% map2(.,qaly_, ~ ({
    tmp = as.matrix(.x) %*% .y
    tmp 
}))

QALY = QALYt %>% map(~sum(.x * disc_h * gen_wcc(params1$omega, method = params1$cycle_correction))) 

# YLD
yld_ = with(params1,(matrix(c(0,
              dw_S1 * Delta_t * (1/r_disc_h_Delta_t) * (1 - exp(-r_disc_h_Delta_t)) ,
              dw_S2 * Delta_t * (1/r_disc_h_Delta_t) * (1 - exp(-r_disc_h_Delta_t)),
              0,
              0),
            dimnames = list(c(
                c(tr_names,ab_names)
            ), c("DW")))
))
yld_ <- 
  with(params1,{
    tx_names %>% map(~({
        if (.x=="A" | .x=="AB") {
          tmp_ <- yld_
          tmp_[2,1] = dw_trtA * Delta_t * (1/r_disc_h_Delta_t) * (1 - exp(-r_disc_h_Delta_t))
          tmp_
      } else yld_
    }))
  })

YLDt <- trace1 %>% map2(.,yld_, ~ ({
    tmp = as.matrix(.x) %*% .y
    tmp 
}))

YLD = YLDt %>% map(~sum(.x*  disc_h * gen_wcc(params1$omega, method = params1$cycle_correction)))

# YLL

new_deaths_from_disease <- 
    map(trace1,~({
        c(0,diff(.x[,"DS"]))
    })) 
   
remaining_life_expectancy <- 
    with(params1,(1/r_disc_h) * (1 - exp(-r_disc_h * fExR(ages_trace))))
    
YLLt <- 
    new_deaths_from_disease %>% map(~(.x * remaining_life_expectancy ))

YLL <- 
    YLLt %>% map(~(sum(.x * disc_h * gen_wcc(params1$omega,method = params1$cycle_correction))))

DALY <- 
    map2(YLL,YLD,~(.x + .y))

# QALY-Like DALY
qaly_daly_ = with(params1,(matrix(c(0,
              dw_S1 * Delta_t * (1/r_disc_h_Delta_t) * (1 - exp(-r_disc_h_Delta_t)) ,
              dw_S2 * Delta_t * (1/r_disc_h_Delta_t) * (1 - exp(-r_disc_h_Delta_t)),
              0,
              1) * Delta_t,
            dimnames = list(c(
                c(tr_names,ab_names)
            ), c("UW")))
))
qaly_daly_ <- 
  with(params1,{
    tx_names %>% map(~({
        if (.x=="A" | .x=="AB") {
          tmp_ <- qaly_daly_
          tmp_[2,1] = dw_trtA * Delta_t * (1/r_disc_h_Delta_t) * (1 - exp(-r_disc_h_Delta_t))
          tmp_
      } else qaly_daly_
    }))
  })

QALY_DALYt <- trace1 %>% map2(.,qaly_daly_, ~ ({
    tmp = as.matrix(.x) %*% .y
    tmp 
}))

QALY_DALY = QALY_DALYt %>% map(~sum(.x * disc_h * gen_wcc(params1$omega, method = params1$cycle_correction))) 

# Costs
cost_ = with(params1,(matrix(c(c_H,
              c_S1 ,
              c_S2,
              c_D,
              c_D)*Delta_t,
            dimnames = list(c(
                c(tr_names,ab_names)
            ), c("COST")))
))
cost_ <-
  with(params1, {
    tx_names %>% map( ~ ({
      if (.x == "A") {
        tmp_ <- cost_
        tmp_["S1", 1] = c_S1 + c_trtA
        tmp_["S2", 1] = c_S2 + c_trtA
        tmp_
      } else if (.x == "B") {
        tmp_ <- cost_
        tmp_["S1", 1] = c_S1 + c_trtB
        tmp_["S2", 1] = c_S2 + c_trtB
        tmp_
      } else if (.x == "AB") {
        tmp_ <- cost_
        tmp_["S1", 1] = c_S1 + c_trtA + c_trtB
        tmp_["S2", 1] = c_S2 + c_trtA + c_trtB
        tmp_
      } else cost_
    }))
  }) %>% 
  set_names(params1$tx_names)

COSTt <- trace1 %>% map2(.,cost_, ~ ({
    tmp = as.matrix(.x) %*% .y
    tmp 
}))

COST = COSTt %>% map(~sum(.x * disc_c * gen_wcc(params1$omega, method = params1$cycle_correction))) 

result1 <- cbind(LE, QALY, YLD, YLL, DALY, QALY_DALY, COST) %>%
  as.data.frame() %>%
  mutate_all( ~ as.numeric(.))  %>%
  rownames_to_column(var = "strategy") %>%
  mutate(approach = "Markov Trace") %>% 
  dplyr::select(approach, strategy, LE, QALY,YLD, YLL, DALY, QALY_DALY, COST) 

result1 %>% 
    kable(digits = 3, col.names = c("Approach","Scenario","Life Expectancy (Model)","QALYs","YLDs","YLLs","DALYs","QALY-like DALY","Costs")) %>% 
    kable_styling()

In [ ]:
result1 %>% 
  select(strategy,QALY,COST) %>% 
  arrange(COST) %>% 
  mutate(iQALY = c(0,diff(QALY)),
         iCOST = c(0,diff(COST))) %>% 
  mutate(icer = iCOST / iQALY) %>% 
  filter(is.na(icer) | icer>0) %>% 
  arrange(COST) %>% 
  mutate(iQALY = c(NA,diff(QALY)),
         iCOST = c(NA,diff(COST))) %>% 
  mutate(icer = iCOST / iQALY) %>% 
  bind_rows(result1 %>% 
  select(strategy,QALY,COST) %>% filter(strategy=="A")) %>% 
  kable() %>% 
  kable_styling()

In [ ]:
result1 %>% 
  select(strategy,DALY,COST) %>% 
  arrange(COST) %>% 
  mutate(iDALY = c(0,pmax(0,-diff(DALY))),
         iCOST = c(0,diff(COST))) %>% 
  mutate(icer = iCOST / iDALY) %>% 
  filter(is.na(icer) | icer>0) %>% 
  filter(!is.infinite(icer)) %>% 
  arrange(COST) %>% 
  mutate(iDALY = c(NA,pmax(0,-diff(DALY))),
         iCOST = c(NA,diff(COST))) %>% 
  mutate(icer = iCOST / iDALY) %>% 
  bind_rows(result1 %>% 
  select(strategy,DALY,COST) %>% filter(strategy=="A")) %>% 
  kable() %>% 
  kable_styling()

In [ ]:
result1 %>% 
  select(strategy,QALY_DALY,COST) %>% 
  arrange(COST) %>% 
  mutate(iDALY = c(0,pmax(0,-diff(QALY_DALY))),
         iCOST = c(0,diff(COST))) %>% 
  mutate(icer = iCOST / iDALY) %>% 
  filter(is.na(icer) | icer>0) %>% 
  filter(!is.infinite(icer)) %>% 
  arrange(COST) %>% 
  mutate(iDALY = c(NA,pmax(0,-diff(QALY_DALY))),
         iCOST = c(NA,diff(COST))) %>% 
  mutate(icer = iCOST / iDALY) %>% 
  bind_rows(result1 %>% 
  select(strategy,QALY_DALY,COST) %>% filter(strategy=="A")) %>% 
  kable() %>% 
  kable_styling()

## Approach 2